# Python Decorator
- awesome [video](https://is.gd/BkaFo7)


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

- Useful alias magics

In [ ]:
%alias_magic t timeit -p "-n 10000 -r 3"

In [ ]:
%alias_magic t1 timeit -p "-n 1 -r 1"

In [ ]:
%alias_magic h history -p "-l 10"

## 1. Closure

In [ ]:
import pdb
def outer_func(msg):
    
    def inner_func():
        print("inner_func called")
        print("\t -- Do extra work with my free variables-- ")
        
#         result = some_func(msg)
        result = msg[::-1]
        return result
        
    return inner_func

myfunc = outer_func('hello')
myfunc2 = outer_func('hi')


In [ ]:
myfunc()

In [ ]:
myfunc2()


## 2. Decorator and wrapper for functions

In [ ]:
# def create_points(n_points):
#     return [(i,i) for i in range(n_points)]
# create_points(3)

def display():
    print('Greetings!')
    
def display2(name, age):
    print(f"{name}: {age}")

display()
display2('hayley', 10)

In [ ]:
def decorator(orig_func):
    
    def wrapper(*args, **kwargs):
        print(orig_func.__name__)
        result = orig_func(*args, **kwargs)
        return result
    
    return wrapper

In [ ]:
# decorated = decorator(create_points)
display2 = decorator(display2)
    

In [ ]:
display2('john',13)

In [ ]:
@decorator
def display():
    print('Greetings!')

@decorator
def display2(name, age):
    print(f"{name}: {age}")

display()
display2('john',12)

## 3. Logger decorator

In [ ]:
LOG_DIR = '../log/temp'
import logging
logging.basicConfig() #https://stackoverflow.com/a/4150236

def my_logger(orig_func):

    fname = f'{LOG_DIR}/{orig_func.__name__}.log'

    logger = logging.getLogger(orig_func.__name__)
    logger.setLevel(logging.INFO)
#     pdb.set_trace()
    f_handler = logging.FileHandler(fname)
    f_handler.setLevel(logging.INFO)
    f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    f_handler.setFormatter(f_format)
    logger.addHandler(f_handler)
    logger.info(f'{logger.name} initiated')
    
    def wrapper(*args, **kwargs):        
        logger.info(f'args: {args}, kwargs: {kwargs}')
        return orig_func(*args, **kwargs)
    
    return wrapper


In [ ]:
@my_logger
def display():
    print('Greetings!')

In [ ]:
@my_logger
def display3(name, age):
    print(f"{name}: {age}")

In [ ]:
display()

In [ ]:
display3('hayley', 20)

In [ ]:
display3('matt', 17)

In [ ]:
logging.root.warning("errrr")
logging.root.handlers

In [ ]:
my_logger

---
## python3: `functools.lru_cache`


In [ ]:
def recursive_func(n):
    if n<2:
        return n
    return recursive_func(n-1) + recursive_func(n-2)
        

In [ ]:
from functools import lru_cache

In [ ]:
def forward(x,w):
    return x*w


In [ ]:
a = np.atleast_2d([0,1,0])
w = np.atleast_2d([1,1,1])


In [ ]:
%%t1 x=np.linspace(-1,1,num=1e4);w=np.random.randn(*x.shape)

forward(x,w), forward(x,w,), forward(x,w)

In [ ]:
@lru_cache(maxsize=10)
def decorated_forward(x,w):
    return x*w


In [ ]:
%%t1 x=np.linspace(-1,1,num=1e4);w=tuple(np.random.randn(*x.shape).tolist());x=tuple(x.tolist())

decorated_forward(x,w), decorated_forward(x,w,), decorated_forward(x,w)

In [ ]:
a = np.atleast_2d([1,2,3])

In [ ]:
a = tuple(a)

In [ ]:
tuple(a.tolist())

---
## Measure execution time of a function or a class method
- [src]()

In [ ]:
import time                                                

def timeit(method):

    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        nprint(f'{method.__name__}, {args}, {kw}, {te-ts:.3f}')
        return result

    return timed

In [ ]:
@timeit
def long_method():
    time.sleep(3)


In [ ]:
long_method()

In [ ]:
class TempClass():
    
    def __init__(self, name):
        self.name=name
    
    @timeit
    def process(self):
        print('Doing some process....')
        time.sleep(2)

In [ ]:
myobj = TempClass('test')
myobj.process()

In [ ]:
%debug